In [1]:
from pipeline2 import PipelineLevels, AcquisitionPipeline, DefaultNameHandler, HDF5RichData, HDF5DataStore
from pipeline2.taskgeneration import (AcquisitionTaskGenerator, JSONFileConfigLoader, DefaultFOVSettingsGenerator,
                                      DefaultScanOffsetsSettingsGenerator, DefaultLocationKeeper,
                                      DefaultLocationRemover, DefaultStageOffsetsSettingsGenerator,
                                      SpiralOffsetGenerator, NewestSettingsSelector, NewestDataSelector,
                                      BoundingBoxLocationGrouper, DefaultScanModeSettingsGenerator)
from pipeline2.detection import LegacySpotPairFinder, SimpleSingleChannelSpotDetector, SimpleLegacyFocusHold, SimpleLocationRepeater
from pipeline2.imspector import ImspectorConnection, get_current_stage_coords
from pipeline2.stoppingcriteria import TimedStoppingCriterion


import specpy as sp

import numpy as np
from matplotlib import pyplot as plt


import h5py as h5


#BoundingBoxLocationGrouper()


In [5]:
# plotting params
%matplotlib inline
plt.rcParams['figure.figsize'] = [10, 10]

# path to generic golbead config_path
gb_config_path = 'C:/Users/RESOLFT/Desktop/config_json/goldbeads_overview.json'

# imspector object
im = sp.Imspector()



pll = PipelineLevels('overview', 'detail', 'more_detail')




name_handler = DefaultNameHandler('C:/Users//RESOLFT/Desktop/TEST_GEN/', pll)

# abuse name_handler to get h5 filename
h5file_name = name_handler.get_path([]).replace('.msr', '.h5')

with h5.File(h5file_name, mode='a') as h5fd:
    data_store = HDF5DataStore(h5fd, pll)

    # pipeline
    pl = (AcquisitionPipeline('1')
            .withImspectorConnection(ImspectorConnection(im).withVerbose())
            .withPipelineLevels(pll)
            .withNameHandler(name_handler)
            .withAddedStoppingCondition(TimedStoppingCriterion(600)))
    
    # attach h5 data store
    pl.withDataStorage(data_store)
    
    
    # overview task generator: same settings every time
    atg_overview = (AcquisitionTaskGenerator(pll.overview, 
                    DefaultLocationRemover(
                        JSONFileConfigLoader(['C:/Users/RESOLFT/Desktop/config_json/test_ov_02072018.json'])
                    ),
                    DefaultStageOffsetsSettingsGenerator(SpiralOffsetGenerator().withStart(get_current_stage_coords(im))),
                    DefaultScanModeSettingsGenerator(['xyz'], True),
                    DefaultFOVSettingsGenerator([[5e-5, 5e-5, 1e-5]], [[3e-7, 3e-7, 4e-7]]),
                    DefaultStageOffsetsSettingsGenerator(SimpleLegacyFocusHold(NewestDataSelector(pl, pll.overview)).withVerbose(False))
                )
                .withDelay(1.0))

    # init detector
    detector = SimpleSingleChannelSpotDetector(NewestDataSelector(pl, pll.overview), 2, 2, withRefinement=False, generateStageOffsets=True).withPlotDetections(False).withVerbose(False)

    # detail task generation: settings from last overview + new FOV + grouped detections
    atg_detail = AcquisitionTaskGenerator(pll.detail,
                             DefaultLocationRemover(NewestSettingsSelector(pl, pll.overview)),
                             DefaultScanModeSettingsGenerator(['xyz'], True),
                             DefaultFOVSettingsGenerator([[5e-6, 5e-6, 2e-6]], [[1e-7, 1e-7, 2e-7]]),
                             DefaultStageOffsetsSettingsGenerator(SimpleLocationRepeater(BoundingBoxLocationGrouper(detector, [5e-6, 5e-6, 2e-6]).withVerbose(False), n=2)))

    atg_detail.withDelay(1.0)

    # overview callback: re-add overview
    pl.withCallbackAtLevel(atg_overview, pll.overview)
    # overview callback2: detect & do detail
    pl.withCallbackAtLevel(atg_detail, pll.overview)


    # another detector in detail
    detector2 = SimpleSingleChannelSpotDetector(NewestDataSelector(pl, pll.detail), 2, 2, withRefinement=False).withPlotDetections(False).withVerbose(False)

    # detail task generation: settings from last overview + new FOV + grouped detections
    atg_detail_more = AcquisitionTaskGenerator(pll.more_detail,
                             DefaultLocationRemover(
                                JSONFileConfigLoader(['C:/Users/RESOLFT/Desktop/config_json/test_2dsted_02072018.json',
                                                     'C:/Users/RESOLFT/Desktop/config_json/test_3dsted_02072018.json'], None, False)
                             ),
                             DefaultLocationKeeper(NewestSettingsSelector(pl, pll.overview)),
                             DefaultScanModeSettingsGenerator(['xy', 'xz'], False),
                             DefaultFOVSettingsGenerator([[5e-7, 5e-7, 5e-7]], [[10e-9, 10e-9, 10e-9]]),
                             DefaultScanOffsetsSettingsGenerator(BoundingBoxLocationGrouper(detector2, [5e-7, 5e-7, 5e-7]).withVerbose(False))
                             )

    atg_detail_more.withDelay(1.0)
    # dummy callback: print & plot detections in detail img
    #pl.withCallbackAtLevel(lambda x: print(detector2.get_locations()), pll.detail)

    # overview callback2: detect & do detail
    pl.withCallbackAtLevel(atg_detail_more, pll.detail)

    # call overview atg once to add first measurement to pipeline
    atg_overview(pl)

    # GO
    pl.run()


running acquisition:
stage offsets: [ 0.0014711  -0.00249828  0.00187054]
scan offsets: [ 0.  0.  0.]
C:/Users//RESOLFT/Desktop/TEST_GEN/da3b3f7e212c7e157757e1cd23ad89ac_overview_0.msr
running acquisition:
stage offsets: [ 0.00145235 -0.00247353  0.00187194]
scan offsets: [ 0.  0.  0.]
C:/Users//RESOLFT/Desktop/TEST_GEN/da3b3f7e212c7e157757e1cd23ad89ac_overview_0_detail_0.msr
0
0
running acquisition:
stage offsets: [ 0.00145235 -0.00247353  0.00187194]
scan offsets: [ 0.  0.  0.]
C:/Users//RESOLFT/Desktop/TEST_GEN/da3b3f7e212c7e157757e1cd23ad89ac_overview_0_detail_1.msr
0
0
running acquisition:
stage offsets: [ 0.00145805 -0.00249663  0.00187194]
scan offsets: [ 0.  0.  0.]
C:/Users//RESOLFT/Desktop/TEST_GEN/da3b3f7e212c7e157757e1cd23ad89ac_overview_0_detail_2.msr
0
0
running acquisition:
stage offsets: [ 0.00145805 -0.00249663  0.00187194]
scan offsets: [ 0.  0.  0.]
C:/Users//RESOLFT/Desktop/TEST_GEN/da3b3f7e212c7e157757e1cd23ad89ac_overview_0_detail_3.msr
0
0
running acquisition:
st

In [ ]:
#detector2.get_locations()
from spot_util import pair_finder_inner, detect_blobs, focus_in_stack

dta = NewestDataSelector(pl, pll.detail).get_data()
img = dta.data[0][0][0,:,:,:]

detect_blobs(img, [3,3,1], 0.00, False)

In [ ]:
import json
im = sp.Imspector()
par = im.active_measurement().parameters('Measurement/axes/num_synced')


im.active_measurement().set_parameters('Measurement/axes/num_synced', 1)
print(json.dumps(par, indent=1))
#print(json.dumps(par['Measurement']['axes']['scan_axes'], indent=2))

#mf = 'xy'
#par['ExpControl']['scan']['range']['mode'] = gen_mode_flag(mf)

#par['Measurement']['axes']['scan_axes'] = ['ExpControl {}'.format(mf[i].upper()) if i < len(mf) else "None" for i in range(4)]

#im.active_measurement().set_parameters('', par)

In [ ]:
modes = {
    'xyz' : 528,
    'xyt' : 784,
    'x' : 1360,
    'y' : 1361,
    'z' : 1362,
    't' : 1363,
    'xy' : 1296,
    'yx' : 1281,
    
    
    
    
    
}

In [ ]:
int('001100000001', 2)

In [ ]:
def gen_mode_flag(mode_str):

    _mode_vals = {
    'x' : 0,
    'y' : 1,
    'z' : 2,
    't' : 3
    }
    
    if len(mode_str) > 4:
        return None
    res = 0
    for _ in range(3 - len(mode_str)):
        print(res)
        res = (res + 1) << 2
        res = (res + 1) << 2
    for i, c in enumerate(reversed(mode_str)):
        res = res << 2
        res = res + _mode_vals[c]
        if not i == len(mode_str) - 1:
            res = res << 2
    return res

gen_mode_flag('xy')

In [ ]:
from skimage.feature import peak_local_max

peak_local_max(np.array([[1,0,0,0], [0,0,0,1]]), min_distance=1)